In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)


data = pd.read_csv('/content/banking.csv') #Subcription data, want to predict if a customer will subcribe
print(data.head(0))
#data['education'].unique()

#This data contains 3 instances of "Basic" education, it has been combined into one
data['education']=np.where(data['education'] =='basic.9y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.6y', 'Basic', data['education'])
data['education']=np.where(data['education'] =='basic.4y', 'Basic', data['education'])

#Ratio of output values - highly uneven
count_0 = len(data[data['y'] == 0]) #Number of 0 subcriptions
count_1 = len(data[data['y'] == 1]) #Number of 1 subcriptions
percent_0 = np.round(count_0/(count_0 + count_1)*100,2)
percent_1 = np.round(count_1/(count_0 + count_1)*100,2) 
# plt.figure(1, figsize=(7,7))
# sns.countplot(x='y', data=data)
# plt.title("Number of Yes/No subscriptions")

#Data Exploration
(data.groupby('age').mean())
(data.groupby('y').mean())
(data.groupby('job').mean())
(data.groupby('education').mean())
# plt.figure(2, figsize=(5,5))
# pd.crosstab(data.job, data.y).plot(kind = 'bar') #plotting percetage of subcriptions per job
# pd.crosstab(data.education, data.y).plot(kind = 'bar') #plotting percetage of subcriptions per education level
# pd.crosstab(data.marital, data.y).plot(kind = 'bar') #plotting percetage of subcriptions per age

#Create dummy variables- making training data
cat_vars=data[['job','marital','education','default','housing','loan','contact','month','day_of_week','poutcome']]
cat_list = pd.get_dummies(cat_vars, drop_first=True)
data=data.join(cat_list)
data=data[[i for i in data if i not in cat_vars]]
y=data[['y']]
x=data[[i for i in data if i not in y]]

#Since yes/no ratio is highly unbalance, use SMOTE to sample the no subscriptions
# X = x.loc[:, x.columns != 'y']
# y = y.loc[:, y.columns == 'y']
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
columns = X_train.columns
os_data_X,os_data_y=smote.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['y'])


# print("length of oversampled data is ",len(os_data_X))
# print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['y']==0]))
# print("Number of subscription",len(os_data_y[os_data_y['y']==1]))
# print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==0])/len(os_data_X))
# print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['y']==1])/len(os_data_X))




In [ ]:
#feature selection using RFE
data_final_vars=x.columns.values.tolist()
y=['y']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())


In [104]:
#print(rfe.support_)
#print(rfe.ranking_)
cols = x.columns[np.where(rfe.support_)]
X=os_data_X[cols]
y=os_data_y['y']
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()


Optimization terminated successfully.
         Current function value: 0.545359
         Iterations 7


In [106]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)